In [1]:
# Automatically reload modules
%load_ext autoreload
%autoreload 2

import os
from dbpd.preprocessing_config import IMUPreprocessingConfig
from dbpd.gait_analysis import *
from dbpd.gait_analysis_config import *
from dbpd.imu_preprocessing import *

In [2]:
path_to_data =  '../../../tests/data'
path_to_classifier = os.path.join(path_to_data, '0.classifiers', 'gait')
path_to_sensor_data = os.path.join(path_to_data, '1.sensor_data', 'imu')
path_to_preprocessed_data = os.path.join(path_to_data, '2.preprocessed_data', 'gait')
path_to_extracted_features = os.path.join(path_to_data, '3.extracted_features', 'gait')
path_to_predictions = os.path.join(path_to_data, '4.predictions', 'gait')
path_to_quantification = os.path.join(path_to_data, '5.quantification', 'gait')

# Cell has the tag 'parameters', so it won't overwrite the parameters when running the notebook in tests

### Preprocessing

In [3]:
config = IMUPreprocessingConfig()
preprocess_imu_data(path_to_sensor_data, path_to_preprocessed_data, config)

### Extract gait features

In [4]:
config = GaitFeatureExtractionConfig()
#config.set_sampling_frequency(50)
extract_gait_features(path_to_preprocessed_data, path_to_extracted_features, config)

### Detect gait

In [5]:
config = GaitDetectionConfig()
detect_gait(path_to_extracted_features, path_to_predictions, path_to_classifier, config)

### Extract arm swing features

In [6]:
config = ArmSwingFeatureExtractionConfig()
extract_arm_swing_features(path_to_preprocessed_data, path_to_extracted_features, config)

### Detect arm swing

In [7]:
config = ArmSwingDetectionConfig()
detect_arm_swing(path_to_extracted_features, path_to_predictions, path_to_classifier, config)

### Quantify arm swing

In [8]:
config = ArmSwingQuantificationConfig()
quantify_arm_swing(path_to_extracted_features, path_to_predictions, path_to_quantification, config)